In [1]:
import os, sys, re, pickle, time, json, gzip, math, glob, csv, tqdm
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt

/data/amandyam/anaconda3/lib/python3.7/site-packages/matplotlib/font_manager.py:232: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
def load2df(outputDir='/data/amandyam/penn_data/pickles/allDfs/', datadir='/data/penn/penn_rt_data/'):
    
    flowsheetsDir = os.path.join(datadir, 'flowsheets/')
    labsDir = os.path.join(datadir, 'labs/')
    locationDir = os.path.join(datadir, 'location/')
    marDir = os.path.join(datadir, 'mar/')
    editDir = '/data/penn/edits/'

    print('1. Medications')
    if os.path.exists(os.path.join(outputDir, 'allMars.pkl')):
        print('Loading')
        mars = pickle.load(open(os.path.join(outputDir, 'allMars.pkl'), 'rb'))
    else:
        mars = []
        for file in tqdm.tqdm(glob.glob(marDir+"*.gz")):
            try:
                mars.append(pd.read_csv(gzip.open(file)))
            except:
                print('Error:', file)
                           
        mars = pd.concat(mars)
        pickle.dump(mars, open(os.path.join(outputDir, 'allMars.pkl'), 'wb'))

    print('2. Flowsheets')
           
    if os.path.exists(os.path.join(outputDir, 'flows3.pkl')):
        print('Loading')
        flows1 = pickle.load(open(os.path.join(outputDir, 'flows1.pkl'), 'rb'))
        flows2 = pickle.load(open(os.path.join(outputDir, 'flows2.pkl'), 'rb'))
        flows3 = pickle.load(open(os.path.join(outputDir, 'flows3.pkl'), 'rb'))
        flows = pd.concat([flows1, flows2, flows3])
    else:
        flows = []
        for file in tqdm.tqdm(glob.glob(flowsheetsDir+"*.gz")):
            try:
                flows.append(pd.read_csv(gzip.open(file)))
            except:
                print('Error', file)

        flows = pd.concat(flows)
        tmp = flows.head(90000000)
        pickle.dump(tmp, open('flows1.pkl', 'wb'))
        tmp = flows.head(180000000).tail(90000000)
        pickle.dump(tmp, open('flows2.pkl', 'wb'))
        tmp = flows.tail(91664460)
        pickle.dump(tmp, open('flows3.pkl', 'wb'))
        #pickle.dump(flows, open(os.path.join(outputDir, 'allFlows.pkl'), 'wb'))
        
    print('3. Locations')
    if os.path.exists(os.path.join(outputDir, 'allLocs.pkl')):
        print('Loading')
        locs = []
        locs = pickle.load(open(os.path.join(outputDir, 'allLocs.pkl'), 'rb'))
    else:
        locs = []
        for file in tqdm.tqdm(glob.glob(locationDir+"*.gz")):
            try:
                locs.append(pd.read_csv(gzip.open(file)))
            except:
                print('Error', file)

        locs = pd.concat(locs)
        pickle.dump(locs, open(os.path.join(outputDir, 'allLocs.pkl'), 'wb'))
                      
    print('4. Labs')

    if os.path.exists(os.path.join(outputDir, 'allLabs.pkl')):
        print('Loading')
        labs = []
        labs = pickle.load(open(os.path.join(outputDir, 'allLabs.pkl'), 'rb'))
    else:
        labs = []
        for file in tqdm.tqdm(glob.glob(labsDir+"*.gz")):
            try:
                labs.append(pd.read_csv(gzip.open(file)))
            except:
                print('Error', file) # error on line 14928
                basename=os.path.basename(file)
                file = os.path.join(editDir, basename.rstrip('.gz'))
                labs.append(pd.read_csv(file))

        labs = pd.concat(labs)
        pickle.dump(labs, open(os.path.join(outputDir, 'allLabs.pkl'), 'wb'))
        
    return mars, flows, locs, labs

In [ ]:
mars, flows, locs, labs = load2df()

  0%|          | 0/330 [00:00<?, ?it/s]

1. Medications


 35%|███▌      | 116/330 [02:00<03:43,  1.05s/it]/data/amandyam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (7,15) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
 75%|███████▍  | 246/330 [04:15<01:27,  1.05s/it]/data/amandyam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
 95%|█████████▍| 312/330 [05:27<00:19,  1.10s/it]/data/amandyam/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
100%|██████████| 330/330 [05:47<00:00,  1.00it/s]


In [ ]:



def paramLists():
    
    ventilation = {'none': ['none (room air)'], 
               'noninvasive': ['nasal cannula', 'humidified nasal cannula', 'high flow nasal cannula',
                               'face tent', 'simple mask', 'venturi mask', 'aerosol mask', 'nasal mask',
                               'partial non-breather mask', 'non-breather mask',
                               'cpap/bipap', 'NIPPV', 'other (comment)', 'sipap', 'blow-by', 'oxyhood'],
               'invasive_wean': ['tracheal collar', 'high flow tracheal collar', 'T piece'], 
               'invasive_vent': ['ventilator', 'jet ventilator', 'oscillator (HFOV)']}
    
    characteristics = {'weight': ['EVS weight kg CAL F121'], 'height': ['EVS height NU'], 
                       'airway_size': ['as airway size'], 'airway_type': ['as airway type']}

    vent_management = {'hob30': ['EVS hob 30 deg'],'vent_mode': ['EVS resp Vent Mode NU'], 
                       'vent_set_rate': ['EVS Resp Vent Set Rate NU'],'tidal_volume_set': ['EVS Resp Vent Set TV NU'],
                       'vent_sensitivity': ['AS penn vent sensitivity lpm NU'],'oxygen_device': ['OXYGENDEVICE1'],
                       'vent_id': ['AS vent id FT'], 'fio2_percent': ['EVS resp vent FiO2% NU', 'EVS o2 fio2 NU 0D4B'],
                       'ventilator_peep': ['EVS resp vent peep NU']} #'o2_delivery_method': [],

    vent_outcomes = {'spontbreath_eval': ['AS spontbreath eval'], 'spontbreath_tolerance': ['AS spontbreath trial tolerance'], 
                     'spontbreath_adverse': ['AS spontbreath adv effect'],
                     'sbt_reason_not_met': ['SBT reason not met'],'sbt_time_began': ['SBT time began'],
                     'sbt_pass_fail': ['SBT pass fail'],'sbt_time_calc': ['SBT time calc'],
                     'sbt_time_term': ['SBT time term'],'sbt_not_met_other': ['SBT not met other'],
                     'sbt_ended': ['SBT ended yes no'],'sbt_fail_reasons': ['SBT fail reasons'],
                     'sbt_fail_other': ['SBT fail other'],'ext_reason_not_met': ['EXT reason not met'], 
                     'ext_screen_met': ['EXT screen met'], 'ext_not_met_other': ['EXT not met other'],
                     'extubation_date': ['EVS DATE EXTUB'], 'extubation_type': ['EVS EXTUB TYPE'], 
                     'extubation_time': ['EVS TIME EXTUB']}

    vitals = {'spo2_percent': ['EVS spo2 NU 7913'], 'temp': ['EVS temp fahrenheit CAL', 'EVS temp celsius CAL'],
              'heart_rate': ['EVS hr heart rate NU', 'EVS hr heart rate NU EE0A'],'resp_rate': ['EVS resp rate NU'], 
              'nibp_systolic': ['EVS bp noninvasive systolic NU'],'npbp_diastolic': ['EVS bp noninvasive diastolic NU'],
              #'abp_systolic': ['EVS bp arterial systolic NU 89F8'],'abp_diastolic': ['EVS bp arterial diastolic NU 1CD2'],
              #'cvp': ['EVS cvp mmhg NU 662C'], 'vo_etco': ['VO_ETCO'], 'vo_paop': ['VO_PAOP'],
              #'pap_systolic': ['EVS pap systolic NU 89BD'], 'pap_diastolic': ['EVS pap diastolic NU A7A3'], 
              'minute_volume': ['EVS Resp Vent MV NU'], 'tidal_volume_actual': ['AS penn tidal vol actual NU'],
              'tidal_volume_exhaled': ['RESP penn med exhaled tidal volume machine NU'], 
              'pain_rating': ['EVS pain rating'],
              'glasgow_score': ['EVSC glasgow score CAL 9313'],'rass_scale': ['AS icu rass scale uphs']}
    
    medications = {'hemodynamic': ['norepinephrine', 'epinephrine', 'vasopressin', 'phenylephrine', 'dobutamine',
                                   'dopamine', 'milrinone', 'epoprostenol', 'nitric'], 
                   'paralytic': ['cisatracurium', 'pancuronium'], 
                   'sedation': ['fentanyl', 'hydromorphone', 'lorazepam', 'morphine','dexmedotomidine', 'midazolam', 'propofol']}
    
    labtests = ['Hgb', 'Hct', #'Sodium Level', 'Potassium Lvl', 'Calcium Lvl', 'Anion Gap'
                'Glucose Lvl', 'RBC', 'WBC', 'Platelet', 'Creatinine', 'BUN', 'pH Art', 'pCO2 Art', 'CO2']
    
    return ventilation, characteristics, vent_management, vent_outcomes, vitals, medications, labtests
    
def select_ventedpts(flows, mars, labs, locs):
    
    ventilated_pts = flows[(flows.value=='ventilator')].patient_id.unique()
    flows = flows[flows.patient_id.isin(ventilated_pts)]
    mars = mars[mars.patient_id.isin(ventilated_pts)]
    labs = labs[labs.patient_id.isin(ventilated_pts)]
    locs = locs[locs.patient_id.isin(ventilated_pts)]
    pickle.dump(flows, open(os.path.join(ventDir, 'flows.pkl'), 'wb'))
    pickle.dump(mars, open(os.path.join(ventDir, 'mars.pkl'), 'wb'))
    pickle.dump(labs, open(os.path.join(ventDir, 'labs.pkl'), 'wb'))
    pickle.dump(locs, open(os.path.join(ventDir, 'locs.pkl'), 'wb'))
           
def getTableSubsets(flows=None, mars=None, locs=None, labs=None):
    
    ventDir = '/data/np6/pennV3/pickles/ventPts/'
    if flows is None:
        flows = pickle.load(open(os.path.join(ventDir, 'flows.pkl'), 'rb'))
        mars = pickle.load(open(os.path.join(ventDir, 'mars.pkl'), 'rb'))
        labs = pickle.load(open(os.path.join(ventDir, 'labs.pkl'), 'rb'))
        locs = pickle.load(open(os.path.join(ventDir, 'locs.pkl'), 'rb'))
    
    ventilation, characteristics, vent_management, vent_outcomes, vitals, medications, labtests = paramLists()
    
    print('Labs')
    labTable = labs[labs.ObsTypeMnemonic.isin(labtests)][['patient_id', 'ObsTypeMnemonic', 'DrawnDate', 'Value', 'Range', 
                                                   'AbnormalFlag', 'ResultDate']]
    labTable = labTable[(~labTable.Value.str.contains("\n|\?|>|<|[a-zA-Z]", na=False))]
    #labTable['DrawnDate'] = pd.to_datetime(labTable.DrawnDate)
    #labTable['Value'] = pd.to_numeric(labTable.Value)
    
    print('Locations')
    locTable = locs[['patient_id', 'valid_on', 'service', 'unit', 'room_bed']]
    
    print('Medications')
    allmeds = '|'.join(np.concatenate([i for i in medications.values()]).astype(str))
    medTable = mars[mars.AdministeredMedicationName.str.contains(allmeds,case=False)][['patient_id','AdministeredMedicationName',
                                                                                       'OrderStartDateTime',
                                                                                       'OrderEndDateTime','InfusionRate',
                                                                                       'AdministeredAmount',
                                                                                       'AdministrationStatus',
                                                                                       'AdministrationNotes',
                                                                                       'RouteText','RouteID']]
    
    print('Ventilation')
    ventTable = flows[flows.name=='EVS o2 device 7B8C'][['patient_id', 'ObservationDateTime', 'name', 'value']]
    ventTable['cat'] = 'none'
    for k in ventilation.keys(): ventTable.loc[ventTable.value.isin(ventilation[k]), 'cat'] = k
    ventTable['catNum'] = 0
    for n, i in enumerate(['none', 'noninvasive', 'invasive_wean', 'invasive_vent']):
        ventTable.loc[ventTable.cat==i, 'catNum'] = n

    print('Flow Entries')
    otherFlowEntries = np.concatenate([np.concatenate([i for i in characteristics.values()]).astype(str), 
                                       np.concatenate([i for i in vitals.values()]).astype(str),
                                       np.concatenate([i for i in vent_management.values()]).astype(str),
                                       np.concatenate([i for i in vent_outcomes.values()]).astype(str)])
    flowTable = flows[flows.name.isin(otherFlowEntries)][['patient_id', 'ObservationDateTime', 'name', 'value']]
    #flowTable['ObservationDateTime'] = pd.to_datetime(flowTable.ObservationDateTime)
    
    print('Saving...', end='')
    pickle.dump(([ventTable, flowTable, labTable, medTable, locTable]), open(os.path.join(ventDir, 'allTables.pkl'), 'wb'))
    print('Done')
    
    
    
      
    